## $\textbf{Inferring Musical Time Signature via Apple Watch Logs of Conductor Wrist Motion}$

$\textbf{Author:}\text{ Ryan Burns}$

$\text{When conducting a chamber group or orchestra, a musical conductor uses a designated baton pattern for each possible time signature a musical}$
$\text{score could be written in. Suppose someone were to wear an Apple Watch on their right wrist while conducting, (assume watch IMU data is}$
$\text{logged at 100 Hz). It is not unreasonable to suspect that we could train a machine learning model to predict the time signature corresponding }$
$\text{to particular wrist motion patterns. A small amount of data is studied to prove this concept using }\textit{long short-term memory (LSTM)}\text{ networks.}$
$\text{For a great overview of LSTM and other recurrent units, see: }\textit{https://colah.github.io/posts/2015-08-Understanding-LSTMs/}\text{. While intended }$
$\text{for sequential data (e.g., the quasi-cyclostationary wrist motions of a conductor), we also admit that this is a small amount of training and }$
$\text{validation data, so the generalization performance of these models across a variety of conductors, time signatures, and tempos could prove}$
$\text{quite poor. With generalization performance being a concern, we a couple techniques to hopefully mitigate some amount of overfitting. }$

$\text{We add Gaussian noise to the input signals to randomly vary input to the network, in addition to applying random dropout to a subset of}$
$\text{the connections within the network at a fixed probability. The model is trained on a subset of data and validated on the remaining data.}$

$\text{This specific deep learning architecture is comprised of a single convolutional layer fed to a dense hyperbolic tangent layer for feature,}$
$\text{extraction preceding the long short-term memory layer. The LSTM units make use of both types of dropout available (ordinary and recurrent)}$
$\text{as measures to reduce overfitting. A multidimensional sliding window is applied to the accelerometer, gyroscope, and IMU-fused watch}$
$\text{orientation quaternion at the input layer of the network.}$

$\textbf{Motion Class Labels:}$

$\text{All class labels are defined for a right-handed user. An orchestral conductor varies their baton pattern according to 1 of 4 possible states, }$
$\text{comprised of 3 time signature classes and a resting class (i.e., cessation of baton motion).}$

$\text{0 }\leftrightarrow [1\enspace0\enspace0\enspace0]\leftrightarrow \text{REST}\Longrightarrow\text{conductor has ceased baton motion (no conducting)}$

$\text{1 }\leftrightarrow [0\enspace1\enspace0\enspace0]\leftrightarrow {2/4}\Longrightarrow\text{conductor proceeds with baton motion consistent with a }\mathbf{\genfrac{}{}{0pt}{}{2}{4}}\text{ time signature}$

$\text{2 }\leftrightarrow [0\enspace0\enspace1\enspace0]\leftrightarrow {3/4}\Longrightarrow\text{conductor proceeds with baton motion consistent with a }\mathbf{\genfrac{}{}{0pt}{}{3}{4}}\text{ time signature}$

$\text{3 }\leftrightarrow [0\enspace0\enspace0\enspace1]\leftrightarrow {4/4}\Longrightarrow\text{conductor proceeds with baton motion consistent with a }\mathbf{\genfrac{}{}{0pt}{}{4}{4}}\text{ time signature}$

$\text{For more information on musical time signatures, visit: }\textit{https://en.wikipedia.org/wiki/Time_signature}.$

![title](baton_motion.png)

$\text{The baton patterns for each time signature of interest are depicted diagramatically above. We assume a tech enthusiast conductor who desires}$
$\text{an Apple Watch app/experience for automatic discrimination between the 3 time signatures above, in addition to a catch-all }\textit{at-rest}\text{ state. We }$
$\text{also assume that this conductor would like automatic time-signature inference to be as tempo-agnostic as possible. Be it }\textit{largo}\text{ or }\textit{prestissimo}\text{,}$
$\text{we assume that the tempo of the musical composition of question would not fool the ideal baton pattern classifier. As such, while the amount}$
$\text{of data collected for this analysis is still limited in its size and diversity (i.e., we can assume overfit models), efforts have been made during}$
$\text{data collection to vary the tempo across each time signature's constituent wrist motion observations. The duration (in seconds or measures)}$
$\text{of each time signature's wrist motion subsequence is also varied during collection. We create an aggregated dataset of independent concatenated}$
$\text{collects for supervised learning in the code below.}$

$\textbf{Note On Labeling:}$

$\text{SensorLog labels are recorded in real time using the app's class label buttons for a streaming iPhone. This iPhone logs data concurrently with}$
$\text{an Apple Watch, which also reports its own class labels. Since toggling of the Apple Watch's class labels using the SensorLog UI on the watch }$
$\text{face would interfere with data collection of wrist motion, we use the }\textit{iPhone}\text{ to log wrist motion labels. By time-aligning the iPhone and Apple}$
$\text{Watch streams below (i.e., using POSIX timestamps), we can readily provide wrist motion labels for the Apple Watch motion signals without}$
$\text{interfering with their trajectory as just described. In short, real-time motion labeling is available through dual stream of Apple Watch }$
$\text{iPhone data, where the former provides the motion observations of interest and the latter provides a mechanism for real-time motion labeling.}$

### $\textbf{Import Packages}$

In [1]:
import pandas as pd
from os import getcwd, environ;
from itertools import product as iter_prod;
from matplotlib import pyplot as plt;
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy import array, hstack, argmax, ones, zeros, log10;
from numpy import logical_or, logical_not, expand_dims, flip;
from numpy import  abs, arange, shape, newaxis, sum, flipud;
from numpy import nan_to_num;
from scipy.signal import parzen;

from tensorflow.keras.models import Sequential, Model;
from tensorflow.keras.optimizers import RMSprop;
from tensorflow.keras.metrics import CategoricalAccuracy;
from tensorflow.keras.metrics import Precision, Recall, AUC;
from tensorflow.keras.layers import Dropout, LSTM, Dense;
from tensorflow.keras.layers import Activation, Reshape, Conv1D;
from tensorflow.keras.layers import Input, GaussianNoise;
from sklearn.metrics import confusion_matrix;

from SensorLogUtils import convert_iPhone_units;

environ['KMP_DUPLICATE_LIB_OK']='True'

### $\textbf{Class Label Definitions}$

In [2]:
# Ordinal motion class labels
class_table = pd.DataFrame({
    'REST': {
        'id': 'REST', 
        'description': 'no conducting / baton pattern',
        '1-hot label': [1,0,0,0],
        'ordinal label': 0
    },
    '2/4': {
        'id': '2/4', 
        'description': 'conducting pattern for a 2/4 time signature',
        '1-hot label': [0,1,0,0],
        'ordinal label': 1
    },
    '3/4': {
        'id': '3/4', 
        'description':'conducting pattern for a 3/4 time signature',
        '1-hot label': [0,0,1,0],
        'ordinal label': 2
    },
    '4/4': {
        'id': '4/4', 
        'description':'conducting pattern for a 4/4 time signature',
        '1-hot label': [0,0,0,1],
        'ordinal label': 3
    }
});

# Number of classes
C = 4;

# Print class table
class_table

,REST,2/4,3/4,4/4
id,REST,2/4,3/4,4/4
description,no conducting / baton pattern,conducting pattern for a 2/4 time signature,conducting pattern for a 3/4 time signature,conducting pattern for a 4/4 time signature
1-hot label,"[1, 0, 0, 0]","[0, 1, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 1]"
ordinal label,0,1,2,3


### $\textbf{Specify & Load Dataset}$

In [3]:
################
# Specify data #
################

# Specify local data storage path
data_path = getcwd() + '/data';

# Collect file string ID
collect_IDs = [
    'time_signatures_collect1',
    'time_signatures_collect2'
];

# Build string filename corresponding to collect_ID
collect_files = [
    (data_path + '/labeled_' + collect_ID + '_appleWatch.csv')
for collect_ID in collect_IDs];

#############
# Load data #
#############

# Load and concatenate dataframes for all
# labeled collects in list collect_IDs
df = pd.concat([pd.read_csv(f, 
    error_bad_lines=False,
    warn_bad_lines=False)
    for f in collect_files
],axis=0,ignore_index=True);

# Data fields (column headers)
fields = [fd for fd in df];

# Length of dataframe
N = len(df); # (samples)

### $\textbf{Select IMU Data & 1-Hot Class Labels}$

In [4]:
# Select input signals
x = nan_to_num(array(df[[
    'accelerometerAccelerationX(m/s^2)',
    'accelerometerAccelerationY(m/s^2)',
    'accelerometerAccelerationZ(m/s^2)',
    'motionRotationRateX(rad/s)',
    'motionRotationRateY(rad/s)',
    'motionRotationRateZ(rad/s)'
]]));

# Define 1-hot labels
y = nan_to_num(array(df[[
    
    # List all class headers in table above
    class_name for class_name in class_table
]]));

### $\textbf{Specify Sliding Observation Window Parameters}$

In [5]:
# Number of raw sensor samples (N)
# and their dimensionality (D)
N, D = shape(x);

# Sliding obs. window length
M = 128; # (samples)

# Step size of obs. window
m = 10;  # (samples)

# Linear grid for sliding observation
# window reflecting window length & step
# size specified above, mapped to raw
# sensor samples n = M, M + m, M + 2m, ...
obs_idx = [n for n in range(M,N,m)];

# Define Parzen window to weight observations
w = expand_dims(parzen(M),axis=1);

### $\textbf{Define Observations & Class Labels}$

In [6]:
# Windowed observation sequence input to model
X = array([w * x[(n - M):n,:] for n in obs_idx]);

# Ground truth 1-hot window-resolution class labels
Y = array([
    [1 if c == argmax(sum(w * y[(n - M):n,:],0)) 
    else 0 for c in range(C)] for n in obs_idx]);

### $\textbf{Training Parameters}$

In [7]:
# Fraction of dataset to hold 
# out for model validation
pct_validation = 0.25;

# Total number of obs.
N_obs = len(X);

# Number of training epochs
N_epoch = 5000;

# Number of batches
N_batch = int(N_obs / 1);

### $\textbf{Define Machine Learning Model}$

In [8]:
#####################################################
# Define model architecture w/ Keras functional API #
#####################################################

# Input layer - M x D observation window
input_layer = Input((M,D));

# Add Gaussian noise to M x D input
h0 = GaussianNoise(5e-3)(input_layer);

# Convolutional layer (M x M)
h1 = Conv1D(M,M,activation='relu')(h0);

# Dense hyperbolic tangent activation layer
h2 = Dense(M,activation='tanh')(h1);

# Long short-term memory layer
h3 = LSTM(2 * M,activation='tanh',
    dropout=0.5,
    recurrent_dropout=0.5)(h2);

# Output layer - C x 1 softmax class activation 
output_layer = Dense(C,activation='softmax')(h3);

# Set up Keras Model() instance
model = Model(
    inputs=input_layer,  # Model inputs
    outputs=output_layer # Model outputs
);

###########################
# Define loss & optimizer #
###########################

# Set RMSprop optimization for 
# speed-of-convergence purposes
opt = RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.001,
    epsilon=1e-07,
    name="RMSprop"
);

# Model compilation, using categorical
# cross-entropy error w/ RMSprop
model.compile(
    
    # Error/loss function
    loss='categorical_crossentropy', 
    
    # Use RMSprop
    optimizer=opt,
    
    # List metrics here
    metrics=[
        CategoricalAccuracy(),
        AUC(),
        Precision(),
        Recall()
    ]
);

# Print a summary table
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 6)]          0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 128, 6)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1, 128)            98432     
_________________________________________________________________
dense (Dense)                (None, 1, 128)            16512     
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 510,212
Trainable params: 510,212
Non-trainable params: 0
_______________________________________________________

### $\textbf{Train Machine Learning Model}$

In [9]:
# Model training
model.fit(
    
    # Input dataset
    x=X, # windowed observations
    y=Y, # 1-hot motion class labels
    
    # Batch size
    batch_size=N_batch,
    
    # Number of training epochs
    epochs=N_epoch,
    
    # Print progress
    verbose=1,
    
    # Set aside fraction for validation
    validation_split=pct_validation,
    
    # False for time series
    shuffle=False,
    
    # Other misc params
    sample_weight=None,
    validation_freq=1,
    max_queue_size=10,
    
    # Parallelize job across 2 workers
    workers=2,
    use_multiprocessing=True
);

Train on 6156 samples, validate on 2052 samples
Epoch 1/5000
6156/6156 [==============================] - 4s 727us/sample - loss: 1.3903 - categorical_accuracy: 0.2437 - auc: 0.5061 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.2758 - val_categorical_accuracy: 0.3743 - val_auc: 0.6907 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 1.3341 - categorical_accuracy: 0.3129 - auc: 0.6302 - precision: 0.5000 - recall: 6.4977e-04 - val_loss: 1.2385 - val_categorical_accuracy: 0.4113 - val_auc: 0.7220 - val_precision: 0.9231 - val_recall: 0.0234
Epoch 3/5000
6156/6156 [==============================] - 1s 106us/sample - loss: 1.2572 - categorical_accuracy: 0.4285 - auc: 0.7024 - precision: 0.6689 - recall: 0.0328 - val_loss: 1.1638 - val_categorical_accuracy: 0.4737 - val_auc: 0.7896 - val_precision: 0.7324 - val_recall: 0.0253
Epoch 4/5000
6156/6156 [==============================] - 1s 104us/s

Epoch 30/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 0.3242 - categorical_accuracy: 0.8809 - auc: 0.9827 - precision: 0.8960 - recall: 0.8660 - val_loss: 0.3563 - val_categorical_accuracy: 0.8855 - val_auc: 0.9783 - val_precision: 0.9053 - val_recall: 0.8670
Epoch 31/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 0.3374 - categorical_accuracy: 0.8663 - auc: 0.9801 - precision: 0.8828 - recall: 0.8489 - val_loss: 0.5201 - val_categorical_accuracy: 0.7817 - val_auc: 0.9588 - val_precision: 0.7922 - val_recall: 0.7749
Epoch 32/5000
6156/6156 [==============================] - 1s 109us/sample - loss: 0.4255 - categorical_accuracy: 0.8294 - auc: 0.9684 - precision: 0.8403 - recall: 0.8147 - val_loss: 0.4677 - val_categorical_accuracy: 0.8124 - val_auc: 0.9630 - val_precision: 0.8249 - val_recall: 0.8012
Epoch 33/5000
6156/6156 [==============================] - 1s 109us/sample - loss: 0.4556 - categorical_accuracy: 0.8083 - auc: 0.965

Epoch 59/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 0.2701 - categorical_accuracy: 0.8929 - auc: 0.9864 - precision: 0.9034 - recall: 0.8843 - val_loss: 0.3203 - val_categorical_accuracy: 0.9050 - val_auc: 0.9822 - val_precision: 0.9136 - val_recall: 0.8972
Epoch 60/5000
6156/6156 [==============================] - 1s 109us/sample - loss: 0.2453 - categorical_accuracy: 0.9064 - auc: 0.9892 - precision: 0.9135 - recall: 0.8986 - val_loss: 0.3361 - val_categorical_accuracy: 0.8752 - val_auc: 0.9785 - val_precision: 0.8871 - val_recall: 0.8655
Epoch 61/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 0.2373 - categorical_accuracy: 0.9113 - auc: 0.9889 - precision: 0.9184 - recall: 0.9032 - val_loss: 0.3049 - val_categorical_accuracy: 0.9108 - val_auc: 0.9836 - val_precision: 0.9157 - val_recall: 0.9050
Epoch 62/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 0.2281 - categorical_accuracy: 0.9116 - auc: 0.990

Epoch 88/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.1697 - categorical_accuracy: 0.9383 - auc: 0.9946 - precision: 0.9436 - recall: 0.9329 - val_loss: 0.2823 - val_categorical_accuracy: 0.9147 - val_auc: 0.9839 - val_precision: 0.9197 - val_recall: 0.9094
Epoch 89/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.1779 - categorical_accuracy: 0.9277 - auc: 0.9940 - precision: 0.9327 - recall: 0.9253 - val_loss: 0.3625 - val_categorical_accuracy: 0.8679 - val_auc: 0.9784 - val_precision: 0.8727 - val_recall: 0.8655
Epoch 90/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.1991 - categorical_accuracy: 0.9172 - auc: 0.9925 - precision: 0.9218 - recall: 0.9139 - val_loss: 0.3299 - val_categorical_accuracy: 0.8748 - val_auc: 0.9807 - val_precision: 0.8802 - val_recall: 0.8699
Epoch 91/5000
6156/6156 [==============================] - 1s 110us/sample - loss: 0.2561 - categorical_accuracy: 0.8871 - auc: 0.988

Epoch 117/5000
6156/6156 [==============================] - 1s 113us/sample - loss: 0.1887 - categorical_accuracy: 0.9277 - auc: 0.9929 - precision: 0.9315 - recall: 0.9254 - val_loss: 0.3243 - val_categorical_accuracy: 0.8923 - val_auc: 0.9806 - val_precision: 0.8951 - val_recall: 0.8899
Epoch 118/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.1595 - categorical_accuracy: 0.9383 - auc: 0.9952 - precision: 0.9420 - recall: 0.9347 - val_loss: 0.2789 - val_categorical_accuracy: 0.9206 - val_auc: 0.9841 - val_precision: 0.9266 - val_recall: 0.9167
Epoch 119/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.1482 - categorical_accuracy: 0.9449 - auc: 0.9958 - precision: 0.9485 - recall: 0.9423 - val_loss: 0.3064 - val_categorical_accuracy: 0.9050 - val_auc: 0.9826 - val_precision: 0.9079 - val_recall: 0.9035
Epoch 120/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.1443 - categorical_accuracy: 0.9503 - auc: 0

...

Epoch 175/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.0929 - categorical_accuracy: 0.9704 - auc: 0.9980 - precision: 0.9729 - recall: 0.9690 - val_loss: 0.2808 - val_categorical_accuracy: 0.9279 - val_auc: 0.9840 - val_precision: 0.9305 - val_recall: 0.9259
Epoch 176/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.0923 - categorical_accuracy: 0.9698 - auc: 0.9981 - precision: 0.9710 - recall: 0.9675 - val_loss: 0.3331 - val_categorical_accuracy: 0.9025 - val_auc: 0.9803 - val_precision: 0.9067 - val_recall: 0.9001
Epoch 177/5000
6156/6156 [==============================] - 1s 114us/sample - loss: 0.0992 - categorical_accuracy: 0.9643 - auc: 0.9978 - precision: 0.9667 - recall: 0.9625 - val_loss: 0.2901 - val_categorical_accuracy: 0.9240 - val_auc: 0.9834 - val_precision: 0.9275 - val_recall: 0.9220
Epoch 178/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.1122 - categorical_accuracy: 0.9584 - auc: 0

Epoch 204/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.0795 - categorical_accuracy: 0.9730 - auc: 0.9985 - precision: 0.9752 - recall: 0.9714 - val_loss: 0.2798 - val_categorical_accuracy: 0.9284 - val_auc: 0.9838 - val_precision: 0.9338 - val_recall: 0.9279
Epoch 205/5000
6156/6156 [==============================] - 1s 119us/sample - loss: 0.0793 - categorical_accuracy: 0.9735 - auc: 0.9987 - precision: 0.9747 - recall: 0.9719 - val_loss: 0.3153 - val_categorical_accuracy: 0.9152 - val_auc: 0.9825 - val_precision: 0.9177 - val_recall: 0.9133
Epoch 206/5000
6156/6156 [==============================] - 1s 116us/sample - loss: 0.0838 - categorical_accuracy: 0.9717 - auc: 0.9985 - precision: 0.9731 - recall: 0.9693 - val_loss: 0.2891 - val_categorical_accuracy: 0.9230 - val_auc: 0.9830 - val_precision: 0.9274 - val_recall: 0.9211
Epoch 207/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.0924 - categorical_accuracy: 0.9647 - auc: 0

Epoch 233/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.0705 - categorical_accuracy: 0.9807 - auc: 0.9990 - precision: 0.9821 - recall: 0.9789 - val_loss: 0.2906 - val_categorical_accuracy: 0.9293 - val_auc: 0.9826 - val_precision: 0.9329 - val_recall: 0.9279
Epoch 234/5000
6156/6156 [==============================] - 1s 112us/sample - loss: 0.0678 - categorical_accuracy: 0.9797 - auc: 0.9987 - precision: 0.9814 - recall: 0.9784 - val_loss: 0.2967 - val_categorical_accuracy: 0.9269 - val_auc: 0.9822 - val_precision: 0.9304 - val_recall: 0.9250
Epoch 235/5000
6156/6156 [==============================] - 1s 114us/sample - loss: 0.0686 - categorical_accuracy: 0.9779 - auc: 0.9989 - precision: 0.9795 - recall: 0.9763 - val_loss: 0.2934 - val_categorical_accuracy: 0.9293 - val_auc: 0.9830 - val_precision: 0.9310 - val_recall: 0.9274
Epoch 236/5000
6156/6156 [==============================] - 1s 111us/sample - loss: 0.0660 - categorical_accuracy: 0.9789 - auc: 0

Epoch 262/5000
6156/6156 [==============================] - 1s 114us/sample - loss: 0.0742 - categorical_accuracy: 0.9751 - auc: 0.9990 - precision: 0.9768 - recall: 0.9721 - val_loss: 0.3084 - val_categorical_accuracy: 0.9196 - val_auc: 0.9832 - val_precision: 0.9212 - val_recall: 0.9172
Epoch 263/5000
6156/6156 [==============================] - 1s 116us/sample - loss: 0.0700 - categorical_accuracy: 0.9771 - auc: 0.9988 - precision: 0.9779 - recall: 0.9763 - val_loss: 0.2874 - val_categorical_accuracy: 0.9274 - val_auc: 0.9837 - val_precision: 0.9295 - val_recall: 0.9254
Epoch 264/5000
6156/6156 [==============================] - 1s 94us/sample


KeyboardInterrupt: 

### $\textbf{Metrics vs. Training Epoch}$

In [ ]:
%matplotlib notebook

# Text legend labels for training set curves
train_set_lbl = ('Training Set (' + 
    str(int(100 * (1 - pct_validation))) + '%)');
    
# Text legend labels for validation set curves
val_set_lbl = ('Training Set (' + 
    str(int(100 * (1 - pct_validation))) + '%)');

# New figure
loss_fig = plt.figure(figsize=(9.9,12));

#############
# Loss plot #
#############
ax1 = plt.subplot(5,1,1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Linear grid of training epochs
epochs = arange(0,N_epoch);

# Cross-entropy error (dB) - training set
plt.plot(epochs,10 * log10(
    model.history.history['loss']),
    '.-',c='g',alpha=0.5,
    label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,10 * log10(
    model.history.history['val_loss']),
    '.-',c='k',alpha=0.5,
    label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Cross-Entropy Error vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Error${}_{\varepsilon}$ (dB)',fontsize=14);

# Legend
plt.legend();

#################
# Accuracy plot #
#################
plt.subplot(5,1,2,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['categorical_accuracy'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_categorical_accuracy'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Categorical Accuracy vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Accuracy${}_{\varepsilon}$ ($\times100\%$)',fontsize=14);

# Legend
plt.legend();

##################
# Precision plot #
##################
plt.subplot(5,1,3,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['precision'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_precision'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]); 

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Precision${}_{\varepsilon}$',fontsize=14);

# Legend
plt.legend();

###############
# Recall plot #
###############
plt.subplot(5,1,4,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['recall'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_recall'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Recall vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'Recall${}_{\varepsilon}$',fontsize=14);

# Legend
plt.legend();

############
# AUC plot #
############
plt.subplot(5,1,5,sharex=ax1);

# Add grid to axes
plt.grid(color='k',alpha=0.25);

# Cross-entropy error (dB) - training set
plt.plot(epochs,model.history.history['auc'],
    '.-',c='g',alpha=0.5,label=train_set_lbl);

# Cross-entropy error (dB) - validation set
plt.plot(epochs,model.history.history['val_auc'],
    '.-',c='k',alpha=0.5,label=val_set_lbl);

# Set x-axis limits
plt.xlim([epochs[0],epochs[-1]]);

# Title
plt.title(r'Area Under ROC Curve vs. Training Epoch $\varepsilon$',
          fontsize=16,weight='bold');

# x-axis label
plt.xlabel(r'Epoch $\varepsilon$',fontsize=14);

# y-axis label
plt.ylabel(r'AUC${}_{\varepsilon}$',fontsize=14);

# Legend
plt.legend();

# Optimize layout
plt.tight_layout();

### $\textbf{Pass Entire Dataset (i.e., Train }\cup\textbf{ Validation) to Model}$

In [ ]:
# C x 1 class prediction for entire dataset
Y_hat = model.predict(
    X,                        # Input data
    batch_size=N_batch,       # Batch size
);

# Cast 1-hot class labels as ordinal labels
y_hat = argmax(Y_hat,axis=1); # Predictions
y_true = argmax(Y,axis=1);    # Ground truth

### $\textbf{Confusion Matrix Using Predictions Output Above}$

In [ ]:
#########################
#    Compute Matrix:    #
#########################

# Use the sklearn confusion matrix function
confusion_mat = confusion_matrix(y_true,y_hat,
    labels=[c for c in range(C)],sample_weight=None);

#########################
#   Confusion Plotter:  # - adapated from online example
#########################

def plot_confusion_matrix(C,classes,normalize=False,
            title='Confusion matrix',cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    # Let epsilon > 0 be small...
    epsilon = 1e-7;
    
    # Normalization of confusion matrix C
    if normalize:
        C = C.astype('float') / (epsilon + C.sum(axis=1)[:, newaxis]);

    # Print the matrix C
    print(C);
    
    # Image representation
    ax = plt.gca();   # <-- grab current axes
    C = flipud(C);    # <-- flip horizontally for visualization
    im = ax.imshow(C,cmap=cmap);
    
    # Colorbar
    colorbar(im);
    
    # Title
    plt.title(title,fontsize=20,weight="bold");
    
    # Tick labels
    tick_marks = arange(len(classes));
    plt.xticks(tick_marks, classes, rotation=45,fontweight="bold");
    
    # Overwrite default y-axis limits
    plt.ylim([-0.5,len(classes)-0.5]);

    # Text labels in tiles (i.e., elements)
    fmt = '.2f' if normalize else 'd'
    thresh = C.max() / 2.
    for i, j in iter_prod(range(C.shape[0]), range(C.shape[1])):
        plt.text(j, i, format(C[i, j], fmt),
                 horizontalalignment="center",weight="bold",
                 color="white" if C[i, j] > thresh else "black")

    # x-axis labels
    plt.ylabel('True label',fontsize=16,weight="bold");
    
    # y-axis labels
    plt.xlabel('Predicted label',fontsize=16,weight="bold");
    
    # Set axis layout
    plt.tight_layout();
    
#########################
#    Scale Colorbar:    # - found this online
#########################

def colorbar(mappable):
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar


#########################
#    Visualization:     #
#########################

conf_fig = plt.figure(figsize=(9.9,9.9));
plot_confusion_matrix(
    confusion_mat, 
    classes=[clss for clss in class_table],
    normalize=True,
    title='Confusion Matrix',
    cmap=plt.cm.Greens);